Usage of the package
========

First, some imports

In [1]:
import gzip
import pickle
import random
from collections import namedtuple

from territories import Territory, MissingTreeCache

## Creation of the tree

The first step is to create a tree of known entities. This can be a very compute intensive task, depending on the tree size. That is why, by default, once created, the tree is stored on disk.

Here we will create a very simple tree out of the **tree.txt** file.

In [2]:
Node = namedtuple('Node', ('id', 'parent_id', 'label', 'level'))
split = lambda x: (arg if arg != 'null' else None for arg in x[:-1].split('; '))

try:
    Territory.load_tree()
except MissingTreeCache:
    with open("tree.txt", "r") as file:
        lines = file.readlines()
        stream = ([Node(*split(x) )for x in lines])
        Territory.build_tree(data_stream=stream, save_tree=False)

Then, you can start to create territories from arbitrary territoiral units.

Entities associated territories are represented in an efficient way : if all leaves of a parent node are included in the territory, they are simply replaced by their parent node.

In [3]:
# some node of the tree
print('\n'.join([f"{e.name} | {e.partition_type}" for e in random.sample(Territory.tree.nodes(), 12)]))

Brissay-Choigny | COM
Berméricourt | COM
Saint-Bonnet-de-Salers | COM
Magny-Jobert | COM
Cassaignes | COM
Bassoncourt | COM
Artigue | COM
Marsonnas | COM
Hodenc-l'Évêque | COM
Mantes-la-Jolie | COM
Illartein | COM
Le Dorat | COM


In [4]:
with open("tree_large.gzip", "rb") as file:
    lines = pickle.loads(gzip.decompress(file.read()))

stream = ([Node(*split(x) )for x in lines])
Territory.build_tree(data_stream=stream)

a = Territory.from_names("COM:69123", "COM:93055", "COM:94052")
b = Territory.from_names("COM:27429", "REG:84", "DEP:75")
c = Territory.from_names("COM:38185", "COM:31555", "REG:11")
d = Territory.from_names("COM:33063", "COM:13055", "REG:28")
e = Territory.from_names("COM:35238", "COM:35047", "DEP:27")
f = Territory.from_names("COM:59350", "COM:38442", "REG:53")

In [5]:
Territory.successors("DEP:69")

[Colombier-Saugnieu,
 Toussieu,
 Ternay,
 Solaize,
 Simandres,
 Sérézin-du-Rhône,
 Sathonay-Village,
 Sathonay-Camp,
 Saint-Symphorien-d'Ozon,
 Saint-Priest,
 Saint-Pierre-de-Chandieu,
 Saint-Laurent-de-Mure,
 Saint-Bonnet-de-Mure,
 Rillieux-la-Pape,
 Pusignan,
 Montanay,
 Mions,
 Meyzieu,
 Marennes,
 Jons,
 Jonage,
 Genay,
 Genas,
 Feyzin,
 Décines-Charpieu,
 Corbas,
 Communay,
 Chassieu,
 Chaponnay,
 Yzeron,
 Vourles,
 Villié-Morgon,
 Villeurbanne,
 Ville-sur-Jarnioux,
 Villefranche-sur-Saône,
 Villechenève,
 Vernay,
 Vernaison,
 Vénissieux,
 Vauxrenard,
 Vaux-en-Beaujolais,
 Vaulx-en-Velin,
 Vaugneray,
 Valsonne,
 Tupin-et-Semons,
 Trèves,
 La Tour-de-Salvagny,
 Thurins,
 Thizy-les-Bourgs,
 Theizé,
 Ternand,
 Tassin-la-Demi-Lune,
 Tarare,
 Taponas,
 Taluyers,
 Saint-Vincent-de-Reins,
 Saint-Vérand,
 Saint-Symphorien-sur-Coise,
 Saint-Romain-en-Gier,
 Saint-Romain-en-Gal,
 Saint-Romain-de-Popey,
 Saint-Romain-au-Mont-d'Or,
 Saint-Pierre-la-Palud,
 Sainte-Paule,
 Saint-Nizier-d'Azergu

## Operations on entities

Usual operation on territories works as expected :

In [6]:
# addition

print(a, c)
print(a + c) # This simplify to France

Nogent-sur-Marne|Pantin|Lyon Toulouse|Île-de-France|Grenoble
Toulouse|Île-de-France|Grenoble|Lyon


In [7]:
# substraction

print(a, d)
print(a - d) # only Saint Etienne remains

Nogent-sur-Marne|Pantin|Lyon Normandie|Marseille|Bordeaux
Nogent-sur-Marne|Pantin|Lyon


More importantly, sets operations are also supported

In [8]:
# intersection
print(f"Intersection of {a} and {d} is {a & d}")

# union
print(f"Union of {c} and {f} is {f | c}")

Intersection of Nogent-sur-Marne|Pantin|Lyon and Normandie|Marseille|Bordeaux is {}
Union of Toulouse|Île-de-France|Grenoble and Bretagne|Lille|Saint-Pierre-de-Chartreuse is Saint-Pierre-de-Chartreuse|Bretagne|Île-de-France|Lille|Grenoble|Toulouse


Territorial units may have parents or children, but Territory do not. As a territory may be formed of several territorial units, it has a LCA, a Lowest Common Ancestor.

In [9]:
f.lowest_common_ancestor()

France

You can easily retrieve all ancestors of a territory with the `.ancestors()` method :

In [10]:
a.ancestors()

{Auvergne-Rhône-Alpes,
 France,
 Rhône,
 Seine-Saint-Denis,
 Val-de-Marne,
 Île-de-France}